In [1]:
import torch
import numpy as np

In [39]:
import ScatterX as x

In [48]:
def pytorchScatter(pytorchScatter_self, dim, index, src):
    l = len(index.shape)
    ls = list(range(l))
    p1 = np.array(tuple(ls[:dim])+tuple(ls[(dim+1):]), dtype='int16');
    p0 = np.array(list(range(l)), dtype='int16');
    p = np.array(tuple(ls[1:(dim+1)])+(0,)+tuple(ls[(dim+1):]), dtype='int16');
    u2 = np.zeros(index.shape);
    def f(I):
        u2[tuple(I)] = src[tuple(I)]
    x.traversal(ind.shape,f)
    x.scatterX(pytorchScatter_self, u2, index, p0, p1, p)

In [51]:
#test case 0 
src = torch.tensor([[0.7654, 0.1543, 0.8820, 0.9940, 0.0098],
        [0.2314, 0.9098, 0.1102, 0.4785, 0.5746]]);
torch.zeros(3, 5).scatter_(1, torch.tensor([[0],[3]]), src)

tensor([[0.7654, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2314, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [52]:
#test case 0 pytorch
src = np.array([[0.7654, 0.1543, 0.8820, 0.9940, 0.0098],
        [0.2314, 0.9098, 0.1102, 0.4785, 0.5746]]);
index = np.array([[0],[3]], dtype='int16');
sf = np.zeros((3, 5))
pytorchScatter(sf, 1, index, src)
sf

array([[0.7654, 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.2314, 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ]])